In [80]:
import os
os.chdir('/engram/nklab/hossein/recurrent_models/BLT_models')
!pwd

%matplotlib inline 
import matplotlib.pyplot as plt

import datasets
import pandas as pd
import numpy as np
import csv
import torch
from matplotlib.patches import Rectangle
import torchvision
from PIL import Image
from models.cornet import get_cornet_model
import torch.nn as nn
from collections import OrderedDict
#import rsatoolbox
import torch.nn.functional as F

import warnings
warnings.filterwarnings("ignore")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# from analyze_representations import sample_vggface2, calc_rdms, plot_maps, compare_rdms,\
#   plot_recurrent_rdms, reduce_dim, plot_dim_reduction_one, sample_FEI_dataset, \
#   plot_rdm_mds, load_model_path, load_pretrained_models, extract_features, kasper_dataset

/engram/nklab/hossein/recurrent_models/BLT_models


In [81]:
A = torch.ones(4, 32, 224, 224)
conv = nn.Conv2d(32, 1, kernel_size=3, dilation=2, stride=2,padding=0, bias=False)
conv.weight = torch.nn.Parameter(torch.ones_like(conv.weight))
out = conv(A)
print(out.shape)

windows = torch.nn.functional.unfold(A, kernel_size=3, dilation=2, stride=2, padding=0)
print(windows.shape)
windows = windows.reshape(4, 1, 32*9, out.shape[-2], out.shape[-1])
windows_reduced = torch.sum(windows, axis=2)
print(windows_reduced.shape)
print(torch.allclose(out, windows_reduced))

torch.Size([4, 1, 110, 110])
torch.Size([4, 288, 12100])
torch.Size([4, 1, 110, 110])
True


In [82]:
windows.shape

torch.Size([4, 1, 288, 110, 110])

In [83]:
import torch
import torch.nn as nn

# Sample input tensor
input_tensor = torch.randn(1, 3, 28, 28)

# Unfold operation
unfold = nn.Unfold(kernel_size=(3, 3), padding=1, stride=1)
unfolded_tensor = unfold(input_tensor)
unfolded_tensor.shape

torch.Size([1, 27, 784])

In [ ]:

# Convolution layer
conv_layer = nn.Conv2d(in_channels=27, out_channels=64, kernel_size=1)
output_tensor = conv_layer(unfolded_tensor)

# Reshape the output
output_tensor = output_tensor.view(1, 64, 28, 28)

print(output_tensor.shape)

Fold, unfold

In [96]:
kernel_size=(2, 2)
stride=1
unfold = nn.Unfold(kernel_size=kernel_size, stride=stride)


conv_1 = nn.Conv2d(3, 8, kernel_size, stride)
w = conv_1.weight
print(w.shape)

inp = torch.randn(1, 2, 3, 2)
inp_unf = unfold(inp) #.transpose(1, 2)
print(inp_unf.shape)

out_unf = inp_unf.matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
out_unf.shape

torch.Size([8, 3, 2, 2])


In [97]:
inp = torch.randn(1, 2, 3, 2)
inp

tensor([[[[ 0.8428,  0.4029],
          [ 0.3548,  0.3547],
          [-0.6932, -0.3738]],

         [[-0.7694,  2.0244],
          [ 0.0813, -1.8770],
          [ 1.7204,  2.2201]]]])

In [104]:
inp[0,0]

tensor([[ 0.8428,  0.4029],
        [ 0.3548,  0.3547],
        [-0.6932, -0.3738]])

In [98]:
inp_unf = unfold(inp) #.transpose(1, 2)
print(inp_unf.shape)
inp_unf

torch.Size([1, 8, 2])


tensor([[[ 0.8428,  0.3548],
         [ 0.4029,  0.3547],
         [ 0.3548, -0.6932],
         [ 0.3547, -0.3738],
         [-0.7694,  0.0813],
         [ 2.0244, -1.8770],
         [ 0.0813,  1.7204],
         [-1.8770,  2.2201]]])

In [106]:
inp_unf.reshape(1, 2, 4, 2)

tensor([[[[ 0.8428,  0.3548],
          [ 0.4029,  0.3547],
          [ 0.3548, -0.6932],
          [ 0.3547, -0.3738]],

         [[-0.7694,  0.0813],
          [ 2.0244, -1.8770],
          [ 0.0813,  1.7204],
          [-1.8770,  2.2201]]]])

In [107]:
inp

tensor([[[[ 0.8428,  0.4029],
          [ 0.3548,  0.3547],
          [-0.6932, -0.3738]],

         [[-0.7694,  2.0244],
          [ 0.0813, -1.8770],
          [ 1.7204,  2.2201]]]])

In [55]:
out_unf = inp_unf.matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
out_unf.shape

torch.Size([5, 8, 4])

In [56]:
torch.nn.functional.conv2d(inp, w).shape

torch.Size([5, 8, 2, 2])

In [57]:
w.view(w.size(0), -1).t().shape

torch.Size([27, 8])

In [108]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

def position(H, W, is_cuda=True):
    if is_cuda:
        loc_w = torch.linspace(-1.0, 1.0, W).cuda().unsqueeze(0).repeat(H, 1)
        loc_h = torch.linspace(-1.0, 1.0, H).cuda().unsqueeze(1).repeat(1, W)
    else:
        loc_w = torch.linspace(-1.0, 1.0, W).unsqueeze(0).repeat(H, 1)
        loc_h = torch.linspace(-1.0, 1.0, H).unsqueeze(1).repeat(1, W)
    loc = torch.cat([loc_w.unsqueeze(0), loc_h.unsqueeze(0)], 0).unsqueeze(0)
    return loc


def stride(x, stride):
    b, c, h, w = x.shape
    return x[:, :, ::stride, ::stride]

def init_rate_half(tensor):
    if tensor is not None:
        tensor.data.fill_(0.5)

def init_rate_0(tensor):
    if tensor is not None:
        tensor.data.fill_(0.)


class ACmix(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_att=7, head=4, kernel_conv=3, stride=1, dilation=1):
        super(ACmix, self).__init__()
        self.in_planes = in_planes
        self.out_planes = out_planes
        self.head = head
        self.kernel_att = kernel_att
        self.kernel_conv = kernel_conv
        self.stride = stride
        self.dilation = dilation
        self.rate1 = torch.nn.Parameter(torch.Tensor(1))
        self.rate2 = torch.nn.Parameter(torch.Tensor(1))
        self.head_dim = self.out_planes // self.head

        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1)
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1)
        self.conv3 = nn.Conv2d(in_planes, out_planes, kernel_size=1)
        self.conv_p = nn.Conv2d(2, self.head_dim, kernel_size=1)

        self.padding_att = (self.dilation * (self.kernel_att - 1) + 1) // 2
        self.pad_att = torch.nn.ReflectionPad2d(self.padding_att)
        self.unfold = nn.Unfold(kernel_size=self.kernel_att, padding=0, stride=self.stride)
        self.softmax = torch.nn.Softmax(dim=1)

        self.fc = nn.Conv2d(3*self.head, self.kernel_conv * self.kernel_conv, kernel_size=1, bias=False)
        self.dep_conv = nn.Conv2d(self.kernel_conv * self.kernel_conv * self.head_dim, out_planes, kernel_size=self.kernel_conv, bias=True, groups=self.head_dim, padding=1, stride=stride)

        self.reset_parameters()
    
    def reset_parameters(self):
        init_rate_half(self.rate1)
        init_rate_half(self.rate2)
        kernel = torch.zeros(self.kernel_conv * self.kernel_conv, self.kernel_conv, self.kernel_conv)
        for i in range(self.kernel_conv * self.kernel_conv):
            kernel[i, i//self.kernel_conv, i%self.kernel_conv] = 1.
        kernel = kernel.squeeze(0).repeat(self.out_planes, 1, 1, 1)
        self.dep_conv.weight = nn.Parameter(data=kernel, requires_grad=True)
        self.dep_conv.bias = init_rate_0(self.dep_conv.bias)

    def forward(self, x):
        q, k, v = self.conv1(x), self.conv2(x), self.conv3(x)
        scaling = float(self.head_dim) ** -0.5
        b, c, h, w = q.shape
        h_out, w_out = h//self.stride, w//self.stride


# ### att
        # ## positional encoding
        pe = self.conv_p(position(h, w, x.is_cuda))

        q_att = q.view(b*self.head, self.head_dim, h, w) * scaling
        k_att = k.view(b*self.head, self.head_dim, h, w)
        v_att = v.view(b*self.head, self.head_dim, h, w)

        if self.stride > 1:
            q_att = stride(q_att, self.stride)
            q_pe = stride(pe, self.stride)
        else:
            q_pe = pe

        unfold_k = self.unfold(self.pad_att(k_att)).view(b*self.head, self.head_dim, self.kernel_att*self.kernel_att, h_out, w_out) # b*head, head_dim, k_att^2, h_out, w_out
        unfold_rpe = self.unfold(self.pad_att(pe)).view(1, self.head_dim, self.kernel_att*self.kernel_att, h_out, w_out) # 1, head_dim, k_att^2, h_out, w_out
        
        att = (q_att.unsqueeze(2)*(unfold_k + q_pe.unsqueeze(2) - unfold_rpe)).sum(1) # (b*head, head_dim, 1, h_out, w_out) * (b*head, head_dim, k_att^2, h_out, w_out) -> (b*head, k_att^2, h_out, w_out)
        att = self.softmax(att)

        out_att = self.unfold(self.pad_att(v_att)).view(b*self.head, self.head_dim, self.kernel_att*self.kernel_att, h_out, w_out)
        print('v_att', out_att.shape)
        print(att.unsqueeze(1).shape)
        out_att = (att.unsqueeze(1) * out_att).sum(2).view(b, self.out_planes, h_out, w_out)
        print('out_att', out_att.shape)

## conv
        f_all = self.fc(torch.cat([q.view(b, self.head, self.head_dim, h*w), k.view(b, self.head, self.head_dim, h*w), v.view(b, self.head, self.head_dim, h*w)], 1))
        f_conv = f_all.permute(0, 2, 1, 3).reshape(x.shape[0], -1, x.shape[-2], x.shape[-1])
        
        out_conv = self.dep_conv(f_conv)

        return self.rate1 * out_att + self.rate2 * out_conv

In [109]:
mix_conv = ACmix(64, 32)

In [110]:
input_tensor = torch.randn(1, 64, 14, 14)

mix_conv(input_tensor).shape

v_att torch.Size([4, 8, 49, 14, 14])
torch.Size([4, 1, 49, 14, 14])
out_att torch.Size([1, 32, 14, 14])


torch.Size([1, 32, 14, 14])

In [145]:
in_channels = 64
out_channels = 32
kernel_size=7 #(7, 7)
stride=1
unfold = nn.Unfold(kernel_size=kernel_size, stride=stride, padding=3)


conv_1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=3)
w = conv_1.weight
print(w.shape)

inp_unf = unfold(input_tensor) #.transpose(1, 2)
print(inp_unf.shape)

inp_unf = inp_unf.transpose(1, 2) 
w = w.view(w.size(0), -1).t()

# out_unf = inp_unf.matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
# out_unf.shape

torch.Size([32, 64, 7, 7])
torch.Size([1, 3136, 196])


In [146]:
inp_unf.shape, w.shape

(torch.Size([1, 196, 3136]), torch.Size([3136, 32]))

In [153]:
out_unf = inp_unf.unsqueeze(-1) * w
out_unf.shape

torch.Size([1, 196, 3136, 32])

In [154]:
out_unf = out_unf.permute(0, 3, 2, 1)
out_unf = out_unf.reshape(1*4, 8, 64, 49, 14, 14).sum(2)
out_unf.shape

torch.Size([4, 8, 49, 14, 14])

In [140]:
conv_unf = out_unf.sum(2).transpose(1, 2).reshape(1, 32, 14, 14)

In [143]:
diff = conv_1(input_tensor) - conv_unf
print(torch.max(conv_unf), torch.min(conv_unf))
print(torch.max(diff), torch.min(diff))

tensor(1.7221, grad_fn=<MaxBackward1>) tensor(-2.0759, grad_fn=<MinBackward1>)
tensor(0.0165, grad_fn=<MaxBackward1>) tensor(-0.0174, grad_fn=<MinBackward1>)


In [144]:
3136/49

64.0